In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [2]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 3

In [4]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/additive/combosciplex/adata_ood_with_predictions_{split}.h5ad")

In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [7]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [8]:
adata_pred_test = adata_train
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test)
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].X.toarray()
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [9]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [10]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [11]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [12]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [14]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8538484147616795,
 'decoded_ood_sinkhorn_div_1': 339.4088701520647,
 'decoded_ood_sinkhorn_div_10': 305.2264142717634,
 'decoded_ood_sinkhorn_div_100': 49.541185651506694,
 'decoded_ood_e_distance': 26.084349006545008,
 'decoded_ood_mmd': 0.06150470035416739}

In [15]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.990503174563249,
 'decoded_test_e_distance': 1.7326015083595696,
 'decoded_test_mmd_distance': 0.009452454202497998}

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8538484147616795,
 'decoded_ood_sinkhorn_div_1': 339.4088701520647,
 'decoded_ood_sinkhorn_div_10': 305.2264142717634,
 'decoded_ood_sinkhorn_div_100': 49.541185651506694,
 'decoded_ood_e_distance': 26.084349006545008,
 'decoded_ood_mmd': 0.06150470035416739}

In [17]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.44901055097579956,
 'encoded_ood_sinkhorn_div_1': 16.89640780857631,
 'encoded_ood_sinkhorn_div_10': 9.906958171299525,
 'encoded_ood_sinkhorn_div_100': 5.984036309378488,
 'encoded_ood_e_distance': 10.831378907127357,
 'encoded_ood_mmd': 0.05829451100102493}

In [18]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.8693430253437587,
 'deg_ood_sinkhorn_div_1': 34.66795894077846,
 'deg_ood_sinkhorn_div_10': 12.288860866001674,
 'deg_ood_sinkhorn_div_100': 4.340018681117466,
 'deg_ood_e_distance': 8.112258587078225,
 'deg_ood_mmd': 0.03690067678689957}

In [19]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.9957934965689977,
 'deg_test_sinkhorn_div_1': 11.266947050889334,
 'deg_test_sinkhorn_div_10': 0.7296160856882731,
 'deg_test_sinkhorn_div_100': 0.13978048165639242,
 'deg_test_e_distance': 0.23858725640090203,
 'deg_test_mmd': 0.007369868457317352}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.990503174563249,
 'decoded_test_e_distance': 1.7326015083595696,
 'decoded_test_mmd_distance': 0.009452454202497998}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8538484147616795,
 'decoded_ood_sinkhorn_div_1': 339.4088701520647,
 'decoded_ood_sinkhorn_div_10': 305.2264142717634,
 'decoded_ood_sinkhorn_div_100': 49.541185651506694,
 'decoded_ood_e_distance': 26.084349006545008,
 'decoded_ood_mmd': 0.06150470035416739}

In [22]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.8693430253437587,
 'deg_ood_sinkhorn_div_1': 34.66795894077846,
 'deg_ood_sinkhorn_div_10': 12.288860866001674,
 'deg_ood_sinkhorn_div_100': 4.340018681117466,
 'deg_ood_e_distance': 8.112258587078225,
 'deg_ood_mmd': 0.03690067678689957}

In [23]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.990503174563249,
 'decoded_test_e_distance': 1.7326015083595696,
 'decoded_test_mmd_distance': 0.009452454202497998}

In [24]:
mean_test_metrics_encoded

{'encoded_test_r_squared': 0.9894989002495107,
 'encoded_test_sinkhorn_div_1': 2.0442960063616433,
 'encoded_test_sinkhorn_div_10': 0.31886033217112225,
 'encoded_test_sinkhorn_div_100': 0.10203484694163005,
 'encoded_test_e_distance': 0.17636510806845998,
 'encoded_test_mmd': 0.00696287020885696}

In [25]:
ood_metrics_encoded

{'Panobinostat+Crizotinib': {'r_squared': 0.32966870069503784,
  'sinkhorn_div_1': 15.071081161499023,
  'sinkhorn_div_10': 8.501304626464844,
  'sinkhorn_div_100': 5.5040435791015625,
  'e_distance': 10.20329419530615,
  'mmd': 0.0405913},
 'Panobinostat+Curcumin': {'r_squared': 0.7438452243804932,
  'sinkhorn_div_1': 10.154277801513672,
  'sinkhorn_div_10': 4.027553558349609,
  'sinkhorn_div_100': 1.02099609375,
  'e_distance': 1.154462362502926,
  'mmd': 0.021306977},
 'Panobinostat+SRT1720': {'r_squared': 0.6198673248291016,
  'sinkhorn_div_1': 9.07442569732666,
  'sinkhorn_div_10': 3.244325637817383,
  'sinkhorn_div_100': 0.6459808349609375,
  'e_distance': 0.5587655163328975,
  'mmd': 0.01937578},
 'Panobinostat+Sorafenib': {'r_squared': -0.49348413944244385,
  'sinkhorn_div_1': 13.365179061889648,
  'sinkhorn_div_10': 7.638408660888672,
  'sinkhorn_div_100': 5.724006652832031,
  'e_distance': 11.002858980823753,
  'mmd': 0.04032762},
 'SRT2104+Alvespimycin': {'r_squared': 0.7265

In [26]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/additive"

In [27]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
